In [2]:
import gentrl
import torch
import pandas as pd
device = torch.device('cpu')

In [3]:
from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol


def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])


def penalized_logP(mol_or_smiles, masked=False, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = logP(mol) - SA(mol) - get_num_rings_6(mol)
    if masked and not mol_passes_filters(mol):
        return default
    return reward

In [4]:
#! wget https://media.githubusercontent.com/media/molecularsets/moses/master/data/dataset_v1.csv

--15:43:04--  https://media.githubusercontent.com/media/molecularsets/moses/master/data/dataset_v1.csv
           => `dataset_v1.csv.1'
Resolving media.githubusercontent.com... 151.101.108.133
Connecting to media.githubusercontent.com|151.101.108.133|:443... connected.
ERROR: Certificate verification error for media.githubusercontent.com: unable to get local issuer certificate
ERROR: certificate common name `www.github.com' doesn't match requested host name `media.githubusercontent.com'.
To connect to media.githubusercontent.com insecurely, use `--no-check-certificate'.
Unable to establish SSL connection.


In [5]:
df = pd.read_csv('dataset_v1.csv')
df = df[df['SPLIT'] == 'train']
df['plogP'] = df['SMILES'].apply(penalized_logP)
df.to_csv('train_plogp_plogpm.csv', index=None)

In [7]:
enc = gentrl.RNNEncoder(latent_size=50)
dec = gentrl.DilConvDecoder(latent_input_size=50)
model = gentrl.GENTRL(enc, dec, 50 * [('c', 20)], [('c', 20)], beta=0.001)
model;

In [8]:
md = gentrl.MolecularDataset(sources=[
    {'path':'train_plogp_plogpm.csv',
     'smiles': 'SMILES',
     'prob': 1,
     'plogP' : 'plogP',
    }], 
    props=['plogP'])

from torch.utils.data import DataLoader
train_loader = DataLoader(md, batch_size=50, shuffle=True, num_workers=1, drop_last=True)

In [9]:
model.train_as_vaelp(train_loader, lr=1e-4)

Epoch 0 :
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
! mkdir -p saved_gentrl

In [ ]:
model.save('./saved_gentrl/')